In [2]:
import numpy as np
import cv2
from ex3_utils import *
from ex2_utils import *
import matplotlib.pyplot as plt

In [22]:
src = "vids/basketball/00000001.jpg"
image = cv2.imread(src)
region = [188.62,321.46,194.43,210.61,231.38,212.54,225.57,323.39]

image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)/255.0
if len(region) == 8:
    x_ = np.array(region[::2])
    y_ = np.array(region[1::2])
    region = [np.min(x_), np.min(y_), np.max(x_) - np.min(x_) + 1, np.max(y_) - np.min(y_) + 1]

window = max(region[2], region[3])

left = max(region[0], 0)
top = max(region[1], 0)

right = min(region[0] + region[2], image.shape[1] - 1)
bottom = min(region[1] + region[3], image.shape[0] - 1)
    
position = (region[0] + region[2] / 2, region[1] + region[3] / 2)
size = (int(region[2] * 1), int(region[3] * 1))

width = size[0]
height = size[1]

#Size is width x height as required for both util functions

print(size)

template, mask = get_patch(image, position, size)
print(template.shape)
G = create_gauss_peak(size, 1)
hanning_window = create_cosine_window(size)

P = (template * hanning_window)

P_hat = np.fft.fft2(P)
P_hat_conjugate = np.conjugate(P_hat)
G_hat = np.fft.fft2(G)

H_hat_conjugate = (G_hat * P_hat_conjugate) / ((P_hat * P_hat_conjugate)+1e-5)

H = np.fft.fft2(H_hat_conjugate).real

R = np.fft.ifft2(H_hat_conjugate * P_hat).real
print(f"P_hat magnitude min: {np.abs(P_hat).min()}, max: {np.abs(P_hat).max()}")
print(f"G_hat magnitude min: {np.abs(G_hat).min()}, max: {np.abs(G_hat).max()}")
cv2.imshow("R", P)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(np.max(R, axis=1))

(43, 113)
(113, 43)
P_hat magnitude min: 0.004331581188979341, max: 722.605777090409
G_hat magnitude min: 0.0013353444987488704, max: 6.283185374416905
[9.99958750e-01 6.06535492e-01 1.35353118e-01 1.11016289e-02
 3.27815698e-04 7.98036720e-06 9.48347798e-06 8.33429238e-06
 1.27203434e-05 1.15626609e-05 1.23643326e-05 1.08545408e-05
 1.01954351e-05 1.22889802e-05 8.81927483e-06 1.30305114e-05
 8.66500930e-06 1.18056083e-05 8.04954624e-06 9.59897613e-06
 8.14200022e-06 1.20770008e-05 1.00076567e-05 1.08921918e-05
 8.15720430e-06 9.07022051e-06 8.40010131e-06 1.06075159e-05
 7.96428854e-06 1.17611935e-05 7.79690669e-06 9.26572841e-06
 8.88757560e-06 9.77381994e-06 1.17053560e-05 9.48504399e-06
 9.71634607e-06 7.68710873e-06 9.43409290e-06 8.23508777e-06
 1.34591877e-05 7.76291880e-06 1.28246889e-05 8.01263763e-06
 8.17376221e-06 1.04470427e-05 1.21108072e-05 9.27398185e-06
 1.11902612e-05 7.23360916e-06 9.49975659e-06 6.22983558e-06
 1.14564179e-05 9.05021486e-06 1.16160259e-05 9.2574163

In [23]:
src = "vids/basketball/00000009.jpg"
image = cv2.imread(src)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)/255.0

template, mask = get_patch(image, position, size)

P = (template * hanning_window)

P_hat = np.fft.fft2(P)
R = np.fft.ifft2(H_hat_conjugate * P_hat).real

x_old = width // 2
y_old = height // 2


y, x = np.unravel_index(R.argmax(), R.shape)
print(y, x)

if x > width / 2:
    x = x - width

if y > height / 2:
    y = y - height

x_new = x_old + x
y_new = y_old + y

print(x_new, y_new)

cv2.imshow("R", R)
cv2.waitKey(0)
cv2.destroyAllWindows()

1 40
18 57
